<a href="https://colab.research.google.com/github/wjdrnqja9/TIL/blob/main/deep_learning/3_3_TF2_Saving_%26_Loading_trained_models_(%2BModelCheckpoint).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br> 

#### 0. Prepare dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
titanic_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/딥러닝/실습 파일/1. Tensorflow (Problem)/titanic_modified.csv")

titanic_target = titanic_df[['Survived']].copy()
titanic_data = titanic_df.copy()

del titanic_data['Survived']

In [7]:
titanic_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,isAlone
0,0,3,0,1.0,0.0,0,3,0.4,0
1,1,1,1,2.0,0.0,2,4,0.4,0
2,1,3,1,1.0,0.0,0,2,0.0,1
3,1,1,1,2.0,0.0,0,4,0.4,0
4,0,3,0,2.0,0.0,0,3,0.0,1


In [6]:
train_data, test_data, train_label, test_label = model_selection.train_test_split(titanic_data, titanic_target,
                                                                                 test_size=0.3,
                                                                                 random_state=0)

In [8]:
enc = preprocessing.OneHotEncoder(categories='auto') 

train_label = enc.fit_transform(train_label).toarray()
test_label = enc.fit_transform(test_label).toarray()

<br> 

#### 1. Train the model

In [9]:
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, activations, initializers, losses, optimizers, metrics

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

In [11]:
model = models.Sequential() 

model.add(layers.Dense(input_dim=8, units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
# model.add(layers.BatchNormalization)
model.add(layers.Activation('elu')) # elu or relu

model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 

model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu'))

model.add(layers.Dense(units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 
model.add(layers.Dropout(rate=0.5))

model.add(layers.Dense(units=2, activation='softmax')) # One-hot vector for 0 & 1

In [12]:
model.compile(optimizer=optimizers.Adam(), 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy]) 

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 256)               2304      
                                                                 
 activation_4 (Activation)   (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 512)               131584    
                                                                 
 activation_5 (Activation)   (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               262656    
                                                                 
 activation_6 (Activation)   (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 256)              

In [15]:
# verbose의 값을 0으로 지정하면 silent mode로 fitting이 진행됩니다.

history = model.fit(train_data, train_label, batch_size=100, epochs=20, validation_split=0.3, verbose=0) 

<br> 

#### 2. Test the model before saving

In [16]:
result = model.evaluate(test_data, test_label)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

9/9 [==============================] - 0s 4ms/step - loss: 0.4307 - categorical_accuracy: 0.8284
loss (cross-entropy) : 0.4307054579257965
test accuracy : 0.8283582329750061


<br> 

#### 3. Save the trained model

In [17]:
# 저장되는 항목 
# - Model의 architecture
# - Model의 모든 weights (Parameter Theta)
# - The state of the optimizer (바로 모델 학습의 재개 가능)

model.save('trained_model.h5') # "Save" the model


# model.save_weights('trained_model.h5') # weights만 따로 저장도 가능함
# model.load_weights('trained_model.h5')

<br> 

#### 4. Load the saved model 

In [18]:
model = models.load_model('trained_model.h5') # "Load" the "model"

# model.load_weights('trained_model.h5') # weights만 따로 불러올 수 있음

In [19]:
result = model.evaluate(test_data, test_label)

print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

9/9 [==============================] - 0s 4ms/step - loss: 0.4307 - categorical_accuracy: 0.8284
loss (cross-entropy) : 0.4307054579257965
test accuracy : 0.8283582329750061


<hr>

<br>

### Appendix) Save the model while training (+ Keras Callbacks API) 

<br>

#### Keras Callbacks API (Keras 콜백 함수)를 사용하는 이유 
<br>

- 모델의 **학습(fit)이 시작된 이후**에는 **학습 완료 전까지 사람이 컨트롤할 수 있는 것이 없음**
- 이를 해결하기 위한 도구가 Keras의 콜백 함수 (프로그래밍 전반에서의 콜백 함수 정의 : https://j.mp/3ibaAT4)
- ex) 학습 중 Learning rate 값을 변화시키기 / 학습 중 일정 시간 성능 개선이 없을 경우 학습 조기 종료 / 학습 중 모델 중간 저장 등
- Keras Callbacks API 공식 문서 : https://keras.io/api/callbacks/

<br>

#### Keras Callback examples
<br>

- **ModelCheckpoint** (학습 중간 저장) : https://deep-deep-deep.tistory.com/53 + 아래 코드 참고
- **EarlyStopping** (학습 조기 종료) : https://deep-deep-deep.tistory.com/55
- **ReduceLROnPlateau** (학습율 자동 조절) : https://deep-deep-deep.tistory.com/56 (Plateau란? @ https://j.mp/3B56FzJ)

In [ ]:
# 다양한 ModelCheckpoint 적용 예제 @ https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko#훈련하는_동안_체크포인트_저장하기
# 아래 Parameter들의 상세 설명 @ https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint

tf.keras.callbacks.ModelCheckpoint( 
    filepath, # 모델 저장 경로, if '{epoch:02d}-{val_loss:.5f}.h5' -> ex) 01(2자리 epoch 수)-0.39121(해당 epoch val_loss값).h5 
    monitor = 'val_loss', # 'loss', 'val_loss', 'accuracy', etc.
    verbose = 0, # 0 or 1
    save_best_only = False, # True : monitor 중인 지표 기준 가장 좋은 모델 저장 or False : 하단 save_freq 기준 주기적 저장
    save_weights_only = False, # True == model.save_weights(filepath) or False == model.save(filepath) 
    mode = 'auto', # 'auto', 'min', 'max'
    save_freq = 'epoch', # 'epoch' or integer(== # of batches) 
    save_freq = 5 * batch_size # == saves the model's weights every 5 epochs (variable 'batch_size' should be set already)
)

In [ ]:
model = models.Sequential() 
model.add(layers.Dense(input_dim=8, units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) # elu or relu
model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 
model.add(layers.Dense(units=512, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu'))
model.add(layers.Dense(units=256, activation=None, kernel_initializer=initializers.he_uniform())) 
model.add(layers.Activation('elu')) 
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(units=2, activation='softmax')) # One-hot vector for 0 & 1

model.compile(optimizer=optimizers.Adam(), 
              loss=losses.categorical_crossentropy, 
              metrics=[metrics.categorical_accuracy]) 

In [ ]:
# 모델 체크포인트 파일(중간 저장 모델)을 저장할 경로 설정 
checkpoint_path = 'saved_models/titanic_4-layer_elu.h5' # 필히 saved_models 폴더를 먼저 만들어줘야 합니다

# "ModelCheckpoint" 콜백함수 객체 생성
callback_checkpoint = tf.keras.callbacks.?(filepath=checkpoint_path, 
                                                         monitor='val_loss', # default
                                                         ?=True, # "Save" the "best" model "only"
                                                         verbose=0)

In [ ]:
# 콜백함수를 호출하며 모델 학습 진행 
history = model.fit(train_data, train_label, 
                    batch_size=100, epochs=100, validation_split=0.3, verbose=0,
                    callbacks=[?]) # 콜백 함수 추가

In [ ]:
# 마지막 학습 완료 시점을 기준으로 한 모델 성능

result = model.evaluate(test_data, test_label)
print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

In [ ]:
# 가장 낮은 Validation loss를 기준으로 한 (저장된) 모델 성능

model = models.?('saved_models/titanic_4-layer_elu.h5') # "Load" the "model"

result = model.evaluate(test_data, test_label)
print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])